# Data base visualization

<br> This module is destined to import images and labels from our sql dataset 
<br> It will use the sqlite library in python to do the necessaries requests  

In [123]:
import sqlite3 as sql
import random
import numpy as np
from PIL import Image
import requests
from io import BytesIO as bio
import os
import keras
import ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
from IPython.display import display, Image

### 1- Creating the labels of our database : #

In [101]:
media_l = ['media_comic', 'media_3d_graphics', 'media_vectorart', 'media_graphite', 'media_pen_ink', 'media_oilpaint', 'media_watercolor']
emotions_l = ['emotion_happy', 'emotion_scary', 'emotion_gloomy', 'emotion_peaceful']
content_l = ['content_building', 'content_flower', 'content_bicycle', 'content_people', 'content_dog', 'content_cars', 'content_cat', 'content_tree', 'content_bird']

### 2- Introducing a couple of image-processing functions :

In [102]:
def load_image(url = ''):
    try:
        reponse = requests.get(url)
        img = Image.open(bio(reponse.content))
        return img
    except:
        return load_image(url = 'https://rhyshonemusic.files.wordpress.com/2013/03/munch-scream-by-pastiche.jpg')

def show_image(img):
    img.show()
    return

def preprocess(x):
    return np.array(x)

### 3- Introducing the data_base class :

In [103]:
class data_base:
    
    def __init__(self, file_path = 'data.sqlite'):
        self.file_path = file_path
        self.mids = []
        self.classes = {0: 'content_building', 1:'emotion_happy', 2:'content_flower',
                    3:'content_bicycle', 4:'media_comic', 5:'content_people',
                    6:'media_3d_graphics',7:'content_dog',8:'media_vectorart',9:'emotion_scary',
                    10:'emotion_gloomy', 11: 'media_graphite', 12: 'emotion_peaceful', 13: 'media_pen_ink',
                    14:'content_cars', 15:'media_oilpaint', 16: 'content_cat', 17: 'content_tree', 18: 'content_bird',19:'media_watercolor'}
        self.classes_labels = {self.classes[i]:i for i in self.classes}

        self.classes_emotions = {0:'emotion_happy',1:'emotion_scary',2:'emotion_gloom',3: 'emotion_peaceful'}

        self.classes_labels_emotions = {self.classes_emotions[i]:i for i in self.classes_emotions}

        self.classes_media = {0:'media_comic',1:'media_3d_graph',2:'media_vectorart',
        3: 'media_graphite',4: 'media_pen_ink' ,5:'media_oilpaint',6:'media_watercolor'}

        self.classes_labels_media = {self.classes_media[i]:i for i in self.classes_media}

    def get_image(self,mid): ## Give the image identifier (MID) and returns the link to the image
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
                SELECT src FROM MODULES
                INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
                where MODULES.mid = ?
        ''',(mid,))
        result = cursor.fetchall()
        if(len(result) == 0):
            print("############# MID = ", mid," DOES NOT EXIST #########################")
            return (result)
        return (result[0][0])

    def get_label(self,mid): ## Gives the image identifier (MID) and returns the image's labels
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
                SELECT *  FROM AUTOMATIC_LABELS WHERE MID = ?
        ''',(mid,))
        result = cursor.fetchall()
        result = list(result[0])
        labels = []
        #print(result)
        for i in result[1:]:
            if(i.lower() == 'positive' and not i.lower() == 'unsure' ):
                labels.append(1)
            else:
                labels.append(0)
        return(labels)

    def get_label_labels(self,mid,label): ## Give the image identifier (MID) and label and returns other labels
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        labels_aux = []
        labels = []
        for i in label:
            cursor.execute('''
                    SELECT "{}"  FROM AUTOMATIC_LABELS WHERE MID = ?
                    '''.format(i.replace('"', '""')),(mid,))
            result = cursor.fetchall()
            labels_aux.append(result[0][0])
            print(result[0])
        #print(result)
        for i in labels_aux:
            if(i.lower() == 'positive' and not i.lower() == 'unsure' ):
                labels.append(1)
            else:
                labels.append(0)
        return(labels)

    def return_classes(self,mid): #Give the MID and it returns the classes it belongs to
        labels = self.get_label(mid)
        classes = []
        for i in range(len(labels)):
            if(labels[i]==1):
                classes.append(self.classes[i])
        return(classes)

    def get_label_emotions(self,mid): ## Gives the image identifier (MID) and returns the emotion labels
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
                SELECT MID, emotion_happy,emotion_scary,emotion_gloomy,emotion_peaceful  FROM AUTOMATIC_LABELS WHERE MID = ?
        ''',(mid,))
        result = cursor.fetchall()
        result = list(result[0])
        #print(result)
        labels = []
        #print(result)
        for i in result[1:]:
            if(i.lower() == 'positive' and not i.lower() == 'unsure' ):
                labels.append(1)
            else:
                labels.append(0)
        return(labels)

    def return_classes_emotions(self,mid): #give the MID and it returns the labels corresponding to the emotions
        labels = self.get_label_emotions(mid)
        classes = []
        for i in range(len(labels)):
            if(labels[i]==1):
                classes.append(self.classes_emotions[i])
        return(classes)

    def get_images_(self,n=100000 ):
        ### creating a cursor to the data base to use SQL commands

        ### Importing data
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
           SELECT MODULES.mid
           FROM MODULES
           INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
           limit ?
        ''', (n,))

        results = cursor.fetchall()
        images = {}
        random.shuffle(results)
        IDs = []
        for img in results:
            IDs.append(img[0])
            #print(img[0],"\n")
        self.mids = IDs
        return IDs

    def get_images(self,n=100000):
        ### creating a cursor to the data base to use SQL commands

        ### Importing data
        proportion = n//5;
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        IDs = []
        for i in self.classes_labels:

            cursor.execute('''
                SELECT MODULES.mid
                FROM MODULES
                INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
                WHERE AUTOMATIC_LABELS."{}" = 'positive'
                order by MODULES.mid
                limit ?
                '''.format(i.replace('"', '""')), (3*proportion,))

            results = cursor.fetchall()
            images = {}
            random.shuffle(results)
            j=0;
            for img in results:
                if(j>proportion):
                    break;
                if(img[0] not in IDs):
                    j=j+1;
                    IDs.append(img[0])
                    #print(img[0])
                #print(img[0],"\n")
        self.mids = IDs
        return(IDs)

    def get_images_emotions(self,n=100000 ):
        ### creating a cursor to the data base to use SQL commands

        ### Importing data
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
           SELECT MODULES.mid, AUTOMATIC_LABELS.emotion_happy, AUTOMATIC_LABELS.emotion_gloomy,  AUTOMATIC_LABELS.emotion_scary, AUTOMATIC_LABELS.emotion_peaceful
           FROM MODULES
           INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
           Where AUTOMATIC_LABELS.emotion_happy = 'positive' OR AUTOMATIC_LABELS.emotion_gloomy = 'positive' OR AUTOMATIC_LABELS.emotion_scary = 'positive'
           OR AUTOMATIC_LABELS.emotion_peaceful = 'positive'
           limit ?
        ''',(n,))

        results = cursor.fetchall()
        images = {}
        random.shuffle(results)
        IDs = []
        i = 0;
        for img in results:
            #label = self.get_label_emotions(img[0])
            #if(1 not in label):
            #    continue;
            IDs.append(img[0])
            #print(img)
            #print(img[0],"\n")
        self.mids = IDs
        return(IDs)

    def get_images_media(self,n=100000 ):
    ### creating a cursor to the data base to use SQL commands

    ### Importing data
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
        SELECT MODULES.mid, AUTOMATIC_LABELS.media_comic AUTOMATIC_LABELS.media_3d_graph,  AUTOMATIC_LABELS.media_vectorart, AUTOMATIC_LABELS.media_graphite
        AUTOMATIC_LABELS.media_pen_ink AUTOMATIC_LABELS.media_oilpaint AUTOMATIC_LABELS.media_watercolor
        FROM MODULES
        INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
        Where AUTOMATIC_LABELS.media_comic = 'positive' OR AUTOMATIC_LABELS.media_3d_graph = 'positive' OR AUTOMATIC_LABELS.media_vectorart = 'positive'
        OR AUTOMATIC_LABELS.media_graphite= 'positive' OR AUTOMATIC_LABELS.media_pen_ink= 'positive' OR AUTOMATIC_LABELS.media_oilpaint= 'positive'
        OR AUTOMATIC_LABELS.media_watercolor= 'positive'
        limit ?
        ''',(n,))
        results = cursor.fetchall()
        images = {}
        random.shuffle(results)
        IDs = []
        i = 0;
        for img in results:
            #label = self.get_label_emotions(img[0])
            #if(1 not in label):
            #    continue;
            IDs.append(img[0])
            #print(img)
            #print(img[0],"\n")
        self.mids = IDs
        return(IDs)

    def get_images_labels(self,n=10000,labels=[]):
        ### creating a cursor to the data base to use SQL commands
        ### Importing data
        proportion = n//len(labels);
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        IDs = []
        for i in labels:
            cursor.execute('''
                SELECT MODULES.mid
                FROM MODULES
                INNER JOIN AUTOMATIC_LABELS on MODULES.mid = AUTOMATIC_LABELS.mid
                WHERE AUTOMATIC_LABELS."{}" = 'positive'
                order by MODULES.mid
                limit ?
                '''.format(i.replace('"', '""')), (5*proportion,))
            results = cursor.fetchall()
            images = {}
            random.shuffle(results)
            j=0;
            for img in results:
                if(j>proportion):
                    break;
                if(img[0] not in IDs):
                    j=j+1;
                    IDs.append(img[0])
                    #print(img[0])
                #print(img[0],"\n")
        self.mids = IDs
        return(IDs)

    def get_label_media(self,mid): ## Gives the image identifier (MID) and returns the link to the image
        conn = sql.connect(self.file_path)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT MID, media_comic, media_3d_graphics, media_vectorart,
            media_graphite, media_pen_ink, media_oilpaint, media_watercolor FROM AUTOMATIC_LABELS WHERE MID = ?
            ''',(mid,))
        result = cursor.fetchall()
        result = list(result[0])
        #print(result)
        labels = []
        #print(result)
        for i in result[1:]:
            if(i.lower() == 'positive' and not i.lower() == 'unsure' ):
                labels.append(1)
            else:
                labels.append(0)
        return(labels)

    def train_test_split(self,n = 1000,test_frac = 0.3, preprocess = preprocess):
        if(len(self.mids) == 0):
            self.get_images(n)
        train_x = []
        train_y = []
        test_x = []
        test_y = []

        for i in range(int((1-test_frac)*n)):
            mid = self.mids[i][0]
            url = self.get_image(mid)
            x = load_image(url)
            x = preprocess(x)
            train_x.append(x)
            train_y.append(self.get_label(mid))

        #print(train_x)
        for i in range(int((1-test_frac)*n),n):
            mid = self.mids[i][0]
            url = self.get_image(mid)
            x = load_image(url)
            x = preprocess(x)
            test_x.append(x)
            test_y.append(self.get_label(mid))

        return(train_x, train_y, test_x, test_y)


### 4- Testing the data base :

We will create a database of a lesser size (called $dbl$) and display the characteristics of the images in them. The images described will be up to the user.

In [121]:
# Creating the data base and the corresponding dataframe
db = data_base()
dbl=db.get_images(100)

def showdf():
    df=pd.DataFrame(columns=['MID','emotions','media labels', 'amount of labels', 'link to image'])
    MID=[]
    E=[]
    M=[]
    L=[]
    N=[]
    for x in dbl:
        MID.append(x)
        E.append(', '.join(db.return_classes_emotions(x)))
        M.append(', '.join(db.return_classes(x)))
        N.append(len(db.return_classes(x)))
        L.append(db.get_image(x))
    df['MID']=MID
    df['emotions']=E
    df['media labels']=M
    df['amount of labels']=N
    df['link to image']=L
    return(df)

showdf()

,MID,emotions,media labels,amount of labels,link to image
0,103624,,"content_building, content_people, media_graphite",3,https://mir-s3-cdn-cf.behance.net/project_modu...
1,66907,,"content_building, content_bicycle",2,https://mir-s3-cdn-cf.behance.net/project_modu...
2,84389,,"content_building, content_tree",2,https://mir-s3-cdn-cf.behance.net/project_modu...
3,32063,,"content_building, content_cars",2,https://mir-s3-cdn-cf.behance.net/project_modu...
4,107194,,content_building,1,https://mir-s3-cdn-cf.behance.net/project_modu...
...,...,...,...,...,...
415,104853,,media_watercolor,1,https://mir-s3-cdn-cf.behance.net/project_modu...
416,109303,,"content_people, media_watercolor",2,https://mir-s3-cdn-cf.behance.net/project_modu...
417,118862,,media_watercolor,1,https://mir-s3-cdn-cf.behance.net/project_modu...
418,86428,,media_watercolor,1,https://mir-s3-cdn-cf.behance.net/project_modu...


In [147]:
# Displaying images and their information

@interact
def show_images(Image_number=range(1,len(df.index))):
    #response = requests.get(file)
    Link=df['link to image'][Image_number]
    display(Image(url=Link, embed=True))
    print("MID :",df['MID'][Image_number])
    print("Labels of this image:",df['media labels'][Image_number])
    print("Link to this image:",df['link to image'][Image_number])

interactive(children=(Dropdown(description='Image_number', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

In [163]:
# Listing images that fit specific criteria

emotions2 = ['All'] + emotions_l
media2 = ['All'] + media_l
content2 = ['All'] + content_l
 
def view_e(e='', dataf=pd.DataFrame()):
    if e=='All': return dataf
    return dataf[dataf['media labels'].str.contains(e)]

def view_m(m='',dataf=pd.DataFrame()):
    if m=='All': return dataf
    return dataf[dataf['media labels'].str.contains(m)]

def view_c(c='',dataf=pd.DataFrame()):
    if c=='All': return dataf
    return dataf[dataf['media labels'].str.contains(c)]

def view(e='',m='',c=''):
    return(view_e(e, view_m(m, view_c(c, df))))


@interact
def show_df(e=emotions2, m=media2, c=content2):
    display(view(e,m,c))
    



interactive(children=(Dropdown(description='e', options=('All', 'emotion_happy', 'emotion_scary', 'emotion_glo…